- use extract_save_features from Labs
- export train.pt and valid.pt to googledrive for easy access (no need to extract features again)
- load extracted features using Tensordataset (torch.load())
- look at feature .py files and see what we need to extract from them to create an ibject (???? correct term?)
- create dataset from extracted features (why?)
- currenty have LabsSolutions modules moved into cwd, but better bet might be to do it ina  forked git repo and run there saves time and code 
- have to look through testing again before running, reason why stopped is because there was no model created/input into !python test_single.py

In [ ]:
#from /LabsSolutions/01-pytorch-object-detection/utils.py
def extract_save_features(loader: torch.utils.data.DataLoader,
                          model: torch.nn.Module,
                          device: torch.device,
                          filename_prefix: str):
     
    with torch.no_grad():
        model.eval()
        batch_idx = 1
        for (inputs, targets) in tqdm(loader):

            inputs = inputs.to(device=device)

            # Compute the forward propagation through the body
            # just to extract the features
            features = model(inputs)

            torch.save(dict([("features", features)] + [(k, v.squeeze()) for (k,v) in targets.items()]),
                       filename_prefix+"{}.pt".format(batch_idx))

            batch_idx += 1
"""The function extract_save_features saves a dictionary. Within the context of this section, the dictionnary that is saved will have :
For the training set :
the key ‘features’ with torch tensor of shape (5717, 512, 7, 7)
the key ‘bboxes’ with torch tensor of shape (5717,4)
the key ‘labels’ with torch tensor of shape (5717)
For the validation set :
the key ‘features’ with torch tensor of shape (5823, 512, 7, 7)
the key ‘bboxes’ with torch tensor of shape (5823,4)
the key ‘labels’ with torch tensor of shape (5823)"""

In [ ]:
# from Labs/01-pytorch-object-detection/utils.py

def extract_save_features(loader: torch.utils.data.DataLoader,
                        model: torch.nn.Module,
                        device: torch.device,
                        filename: str):
    """The function extract_save_features saves a dictionary. Within the context of this section, the dictionnary that is saved will have :

For the training set :
the key ‘features’ with torch tensor of shape (5717, 512, 7, 7)
the key ‘bboxes’ with torch tensor of shape (5717,4)
the key ‘labels’ with torch tensor of shape (5717)
For the validation set :
the key ‘features’ with torch tensor of shape (5823, 512, 7, 7)
the key ‘bboxes’ with torch tensor of shape (5823,4)
the key ‘labels’ with torch tensor of shape (5823)"""
    all_features = []
    all_targets = defaultdict(list)

    with torch.no_grad():
        model.eval()
        for (inputs, targets) in tqdm(loader):

            inputs = inputs.to(device=device)

            # Compute the forward propagation through the body
            # just to extract the features
            all_features.append(model(inputs))

            for k, v in targets.items():
                all_targets[k].append(v)

        for k, v in all_targets.items():
            all_targets[k] = torch.squeeze(torch.cat(v, 0))
        all_features = torch.squeeze(torch.cat(all_features , 0))
    print(all_features.shape)
    print("The features that are saved are {} features maps of size {} x {}, with {} channels".format(all_features.shape[0], all_features.shape[2], all_features.shape[3], all_features.shape[1]))
    for k, v in all_targets.items():
        print("The entry {} have shape {}".format(k, v.shape))

    torch.save(dict([("features", all_features)] + list(all_targets.items())), filename)




In [ ]:
Key Differences:

Filename Handling:

Function 1: Accepts a single filename argument, implying it saves all extracted data (features, bounding boxes, labels) into a single file with that name.
Function 2: Accepts a filename_prefix argument, suggesting it might save data into multiple files, using the prefix to differentiate them (e.g., filename_prefix_features.pt, filename_prefix_bboxes.pt, etc.).
Data Saving Logic:

Function 1: Saves all features, bounding boxes, and labels together in a dictionary and stores it as a single .pt file.
Function 2: Could potentially save them as separate files or using a different dictionary structure, based on a user-defined parameter.
Functionality:

Function 1: Extracts features from a given model and combines them with target data (bboxes, labels). It then saves everything as a single file for use in another notebook.
Function 2: Without the function body, it's unclear about its exact logic for saving features, bounding boxes, and labels. It could save them separately, combined in another form, or incorporate additional flexibility.
In Summary:

The main difference is how they handle the saving of data - using a single file vs. a prefix for multiple files. Additionally, function 2's behavior without its code is less specific, offering some room for differing data handling strategies. This likely affects its application and data usage within other notebooks.